In [21]:
from bs4 import BeautifulSoup
import requests

from fake_useragent import FakeUserAgent

from dotenv import dotenv_values

import re
import csv

import pandas as pd

In [22]:
# Подгрузка данных из файла окружения
config = dotenv_values(".env")

# Обращения
# config[hh_api_name]
# config[hh_api_Client_ID]
# config[hh_api_Client_Secret]

In [23]:
# Основные регулярные выражения для проекта
re_vacancy_id_hh = r'\/vacancy\/(\d+)\?'

# search(re_vacancy, string).group(1)

Необходимо достать в результате

- Название вакансии 
- Название компании 
- З/п
- Формат работы
- Тип занятости 
- Навыки (Если можно достать отдельно)
- Описание вакансии (если есть требования или навыки - то отлично, берем их)
- Время публикации 
- Ссылка на вакансию тоже должна быть у нас

In [30]:
class Rabota1000_Parser:
    # Класс для парсинга вакансий с ресурса Rabota1000.ru
    

    def __init__(self, city:str='russia'):
        self.basic_url = 'https://rabota1000.ru/russia/'
        self.vac_name_list = []
        self.vac_name_list = self.get_vac_name_into_file('vacancy_name_to_pars.txt')
        
        ua = FakeUserAgent()
        headers = {'user-agent':ua.random}

        self._pars()
        self._pars_url_hh()

    def _pars(self):
        with open('pars_link.csv', 'w', newline='') as csv_file:
            names = ['page', 'link']
            file_writer = csv.DictWriter(csv_file, delimiter = ",", lineterminator="\r", fieldnames=names)
            file_writer.writeheader()

        for vac_name in self.vac_name_list:
            print(vac_name)
            try:
                used_url = self.basic_url + vac_name + "/"
                response = requests.get(used_url)
                response.raise_for_status()
                for i in range(1, 51):
                    print(i, end=' ')
                    used_url = f'{self.basic_url}/{vac_name}?p={i}'
                    page = requests.get(used_url)
                    soup = BeautifulSoup(page.text, 'html.parser')

                    # 20 ссылок на одной странице
                    links = [link['href'] for link in soup.findAll('a', attrs={'@click':'vacancyLinkClickHandler'})]
                    links_to_save = [[vac_name, link]for link in links]
                    with open('pars_link.csv', 'a', newline='') as csv_file:
                        writer = csv.writer(csv_file)
                        writer.writerows(links_to_save)

            except HTTPError as exc:
                code = exc.response.status_code
                print(code)
            print()

    def get_vac_name_into_file(self, vac_file_path:str)->list:
        vac_name_list = []
        with open(vac_file_path, mode='r', encoding="utf-8") as file_vac:
            vac_name_list = list(map(lambda x: x.replace('\n', '').replace(' ','+'), file_vac.readlines()))

        return vac_name_list

    def _pars_url_hh(self, url=''):
        ...

parser = Rabota1000_Parser()

data+scientist
1 2 3 4 

NameError: name 'HTTPError' is not defined